# Dev contrastive knowledge assesment notebook

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/CKA_dev.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dependencies

In [1]:
!git clone https://github.com/daniel-furman/Capstone.git

Cloning into 'Capstone'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 287 (delta 129), reused 230 (delta 73), pack-reused 0
Receiving objects: 100% (287/287), 21.09 MiB | 13.12 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [2]:
!pip install -r /content/Capstone/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.5 MB/s eta 0:00:00


## Imports

In [3]:
import os
import numpy as np

import torch
from torch.nn.functional import softmax

from transformers import (
    set_seed,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    T5Tokenizer,
    T5ForConditionalGeneration,
)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dev functions for new models

In [5]:
def probe_flan(model, tokenizer, target_id, context, verbose=False):

    # tokenize context
    tokenized_context = tokenizer(
        context,
        padding="longest",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    ).to(device)

    input_ids = tokenized_context["input_ids"]

    # use model to solicit a prediction
    outputs = model(
        input_ids=input_ids,
        decoder_input_ids=torch.tensor([[0, 32099]], device="cuda:0"),
        output_hidden_states=True,
        return_dict=True,
    )

    # We have batch size of 1, so grab that, then,
    # Take the entire last matrix which corresponds to the last layer
    logits = outputs["logits"][0, -1]

    # convert our prediction scores to a probability distribution with softmax
    probs = softmax(logits, dim=-1)

    probs = probs.detach().cpu().numpy()

    if verbose:
        print(f"\n\tcontext... {context}")
        print(f'\ttokenized_context ids... {tokenized_context["input_ids"]}')
        print(
            f'\tdecoded tokenized_context... {tokenizer.decode(tokenized_context["input_ids"][0])}'
        )
        print(f"\tdecoded target id... {tokenizer.decode([target_id.item()])}")
        print(
            f"\tmost probable prediction id decoded... {tokenizer.decode([np.argmax(probs)])}\n"
        )

    return probs[target_id.item()]


def probe_gpt2(model, tokenizer, target_id, context, verbose=False):

    # tokenize context
    tokenized_context = tokenizer(
        context,
        return_tensors="pt",
    ).to(device)

    input_ids = tokenized_context["input_ids"]

    # grab value
    target_scalar = target_id.detach().cpu().numpy()

    # use model to solicit a prediction
    outputs = model(input_ids=input_ids, output_hidden_states=True, return_dict=True)

    # shape of 50257 which corresponds to the vocab size of GPT
    # every token in GPT's vocab gets a representative prediction from the model
    logits = outputs["logits"][0, -1]
    # convert our prediction scores to a probability distribution with softmax
    probs = softmax(logits, dim=-1)

    probs = list(probs.detach().cpu().numpy())

    if verbose:
        print(f"\n\tcontext... {context}")
        print(f'\ttokenized_context ids... {tokenized_context["input_ids"]}')
        print(
            f'\tdecoded tokenized_context... {tokenizer.decode(tokenized_context["input_ids"][0])}'
        )
        print(f"\tdecoded target id... {tokenizer.decode([target_id.item()])}")
        print(
            f"\tmost probable prediction id decoded... {tokenizer.decode([np.argmax(probs)])}\n"
        )

    # double check weird-ness before accessing prob
    if len(probs) < target_id:
        return None

    # return the likelihood that our stipulated target would follow the context,
    # according to the model
    try:
        return np.take(probs, [target_scalar])[0]

    except IndexError:

        print("target index not in model vocabulary scope; raising IndexError")
        return None


def probe_bert(model, tokenizer, target_id, context, verbose=False):

    # tokenize context
    tokenized_context = tokenizer(
        context,
        padding="longest",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    )

    mask_token_index = torch.where(
        tokenized_context["input_ids"] == tokenizer.mask_token_id
    )[1]

    # use model to solicit a prediction
    logits = model(**tokenized_context.to(device)).logits
    mask_token_logits = logits[0, mask_token_index, :]

    # Convert our prediction scores to a probability distribution with softmax
    probs = torch.squeeze(softmax(mask_token_logits, dim=-1))

    probs = probs.detach().cpu().numpy()

    if verbose:
        print(f"\n\tcontext... {context}")
        print(f'\ttokenized_context ids... {tokenized_context["input_ids"]}')
        print(
            f'\tdecoded tokenize_context... {tokenizer.decode(tokenized_context["input_ids"][0])}'
        )
        print(f"\tmask token id... {tokenizer.mask_token_id}")
        print(f"\tmask token index in context... {mask_token_index}")
        print(f"\tdecoded target id... {tokenizer.decode([target_id.item()])}")
        print(
            f"\tmost probable prediction id decoded... {tokenizer.decode([np.argmax(probs)])}\n"
        )

    return probs[target_id.item()]


In [6]:
# first, write helper to pull a pretrained LM and tokenizer off the shelf
def get_model_and_tokenizer(model_name):
    if "flan" in model_name.lower():
        return T5Tokenizer.from_pretrained(
            model_name
        ), T5ForConditionalGeneration.from_pretrained(
            model_name, load_in_8bit=True, device_map="auto"
        )

    elif ("gpt" in model_name.lower()) or ("opt" in model_name.lower()):
        return AutoTokenizer.from_pretrained(
            model_name
        ), AutoModelForCausalLM.from_pretrained(
            model_name, load_in_8bit=True, device_map="auto"
        )

    elif "bert" in model_name.lower():
        return AutoTokenizer.from_pretrained(
            model_name
        ), AutoModelForMaskedLM.from_pretrained(
            model_name, torch_dtype=torch.float16
        ).to(
            device
        )


# next, write a helper to pull a probe function for the given LM
def get_probe_function(prefix):
    probe_functions = [probe_flan, probe_gpt2, probe_bert]
    for func in probe_functions:
        if prefix.lower() in func.__name__:
            return func


# lastly, write a wrapper function to compare models
def compare_models(model_name_list, input_pairings, verbose):

    """
    Model-wise comparison helper function

    we should be able to do the following:
      * input a set of models we want to evaluate
      * input an expression of interest
      * input a 'true' next-token alonside a false
      * and get an output report that contains..
        * the 'result' ie is true > false
        * the probabilities of both of those values
      * running this method over a large set of positive/negative pairings should result in a large pool of information that can be used to compare model-families
      * we can also look at the relative 'certainty' across different models (at least in orders of magnitude)

    """

    score_dict_full = {}
    score_dict_succinct = {}
    score_dict_summary = {}

    for model_name in model_name_list:
        true_count = 0
        fact_count = 0
        print(f"CKA for {model_name}")
        print("Loading  model...")

        # get proper model and tokenizer
        tokenizer, model = get_model_and_tokenizer(model_name)

        print("Running comparisons...")

        # establish prefix
        prefix = ""
        probe_func = None

        # get correct CKA function
        if ("t5" in model_name.lower()) or ("ul2" in model_name.lower()):
            prefix = "flan"
            probe_func = get_probe_function(prefix)

        elif "gpt" in model_name.lower():
            prefix = "gpt"
            probe_func = get_probe_function(prefix)

        elif "opt" in model_name.lower():
            prefix = "opt"
            probe_func = get_probe_function("gpt")

        elif "roberta" in model_name.lower():
            prefix = "roberta"
            probe_func = get_probe_function("bert")

        elif "bert" in model_name.lower():
            prefix = "bert"
            probe_func = get_probe_function(prefix)

        # iterate over context/entity pairings
        # input_pairings is a dict
        # context is a plain string (since our context's will be unique)
        # and entities is a list containing, in the first slot, the true
        # value for the statement and in the subsequent slots, incorrect information

        for fact_itr, entities_dict in input_pairings.items():

            for counterfact in entities_dict["false"]:

                fact_count += 1

                context = entities_dict["stem"]
                entities = [entities_dict["true"], counterfact]
                entity_count = 0
                p_true = 0.0
                p_false = 0.0

                if prefix == "flan":
                    context += " <extra_id_0>."
                elif prefix == "roberta":
                    context += " <mask>."
                elif prefix == "bert":
                    context += " [MASK]."

                for entity in entities:
                    target_id = None
                    # first find target vocab id
                    # default to the very first token that get's predicted
                    # e.g. in the case of Tokyo, which gets split into <Tok> <yo>,

                    if prefix == "flan":
                        target_id = tokenizer.encode(
                            entity,
                            padding="longest",
                            max_length=512,
                            truncation=True,
                            return_tensors="pt",
                        ).to(device)[0][0]

                    elif prefix == "gpt":
                        target_id = tokenizer.encode(
                            " " + entity, return_tensors="pt"
                        ).to(device)[0][0]

                    elif prefix == "opt":
                        target_id = tokenizer.encode(
                            " " + entity, return_tensors="pt"
                        ).to(device)[0][1]

                    elif prefix == "roberta":
                        target_id = tokenizer.encode(
                            " " + entity,
                            padding="longest",
                            max_length=512,
                            truncation=True,
                            return_tensors="pt",
                        ).to(device)[0][1]

                    elif prefix == "bert":
                        target_id = tokenizer.encode(
                            entity,
                            padding="longest",
                            max_length=512,
                            truncation=True,
                            return_tensors="pt",
                        ).to(device)[0][1]

                    # next call probe function
                    model_prob = probe_func(
                        model, tokenizer, target_id, context, verbose
                    )

                    # lastly, register results
                    if entity_count == 0:
                        p_true = model_prob

                    else:
                        p_false += model_prob

                    entity_count += 1

                p_false /= entity_count - 1

                try:
                    score_dict_full[model_name.lower()].append(
                        {
                            context
                            + " "
                            + f"{entities}": {
                                "p_true": p_true,
                                "p_false": p_false,
                                "p_true - p_false": p_true - p_false,
                                "p_true > p_false": p_true > p_false,
                            }
                        }
                    )
                except KeyError:
                    score_dict_full[model_name.lower()] = [
                        {
                            context
                            + " "
                            + f"{entities}": {
                                "p_true": p_true,
                                "p_false": p_false,
                                "p_true - p_false": p_true - p_false,
                                "p_true > p_false": p_true > p_false,
                            }
                        }
                    ]

                try:
                    score_dict_succinct[model_name.lower()].append(
                        {
                            context
                            + " "
                            + f"{entities}": {
                                "p_true > p_false": p_true > p_false,
                            }
                        }
                    )
                except KeyError:
                    score_dict_succinct[model_name.lower()] = [
                        {
                            context
                            + " "
                            + f"{entities}": {
                                "p_true > p_false": p_true > p_false,
                            }
                        }
                    ]

                if p_true > p_false:
                    true_count += 1

        score_dict_summary[
            model_name.lower()
        ] = f"This model got {true_count}/{fact_count} facts correct"

        print("Done\n")
        del tokenizer
        del model
        torch.cuda.empty_cache()

    return score_dict_full, score_dict_succinct, score_dict_summary

## Test the functions

In [7]:
def main(config):

    set_seed(42)

    score_dict_full, score_dict_succinct, score_dict_summary = compare_models(
        config["models"], config["input_information"], config["verbosity"]
    )

    return score_dict_full, score_dict_succinct, score_dict_summary

### OPT

In [8]:
config = {
    "models": [
        "facebook/opt-125m",  # 125M params
    ],
    "input_information": {
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false":["London","Berlin", "Chicago"]
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false":["Manila","Santiago", "Baghdad"]
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false":["Microsoft","Oracle", "Intel"]
        }
    },
    "verbosity": True
}


In [9]:
score_dict_full, score_dict_succinct, score_dict_summary = main(config)

CKA for facebook/opt-125m
Loading  model...



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Running comparisons...

	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[   2,  133, 2760, 4365,   58,  547,   11]], device='cuda:0')
	decoded tokenized_context... </s>The 2020 Olympics were held in
	decoded target id...  Tokyo
	most probable prediction id decoded...  Tokyo


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[   2,  133, 2760, 4365,   58,  547,   11]], device='cuda:0')
	decoded tokenized_context... </s>The 2020 Olympics were held in
	decoded target id...  London
	most probable prediction id decoded...  Tokyo


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[   2,  133, 2760, 4365,   58,  547,   11]], device='cuda:0')
	decoded tokenized_context... </s>The 2020 Olympics were held in
	decoded target id...  Tokyo
	most probable prediction id decoded...  Tokyo


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[   2,  133, 2760, 4365,   58,  547,   11]], device=

In [10]:
score_dict_full

{'facebook/opt-125m': [{"The 2020 Olympics were held in ['Tokyo', 'London']": {'p_true': 0.1683,
    'p_false': 0.0361328125,
    'p_true - p_false': 0.1322021484375,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Berlin']": {'p_true': 0.1683,
    'p_false': 0.0012950897216796875,
    'p_true - p_false': 0.1670398712158203,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Chicago']": {'p_true': 0.1683,
    'p_false': 0.0007886886596679688,
    'p_true - p_false': 0.16754627227783203,
    'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Manila']": {'p_true': 0.0005836,
    'p_false': 0.00018596649169921875,
    'p_true - p_false': 0.00039768218994140625,
    'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Santiago']": {'p_true': 0.0005836,
    'p_false': 2.2292137145996094e-05,
    'p_true - p_false': 0.0005613565444946289,
    'p_true > p_false': True}},
  {"Operatio

In [11]:
score_dict_succinct

{'facebook/opt-125m': [{"The 2020 Olympics were held in ['Tokyo', 'London']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Berlin']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Chicago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Manila']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Santiago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Baghdad']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Microsoft']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Oracle']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Intel']": {'p_true > p_false': True}}]}

In [12]:
score_dict_summary

{'facebook/opt-125m': 'This model got 9/9 facts correct'}

### RoBERTa

In [13]:
config = {
    "models": [
        "distilroberta-base",  # 82M params
    ],
    "input_information": {
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false":["London","Berlin", "Chicago"]
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false":["Manila","Santiago", "Baghdad"]
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false":["Microsoft","Oracle", "Intel"]
        }
    },
    "verbosity": True
}


In [14]:
score_dict_full, score_dict_succinct, score_dict_summary = main(config)

CKA for distilroberta-base
Loading  model...


Running comparisons...

	context... The 2020 Olympics were held in <mask>.
	tokenized_context ids... tensor([[    0,   133,  2760,  4365,    58,   547,    11, 50264,     4,     2]],
       device='cuda:0')
	decoded tokenize_context... <s>The 2020 Olympics were held in<mask>.</s>
	mask token id... 50264
	mask token index in context... tensor([7])
	decoded target id...  Tokyo
	most probable prediction id decoded...  Tokyo


	context... The 2020 Olympics were held in <mask>.
	tokenized_context ids... tensor([[    0,   133,  2760,  4365,    58,   547,    11, 50264,     4,     2]],
       device='cuda:0')
	decoded tokenize_context... <s>The 2020 Olympics were held in<mask>.</s>
	mask token id... 50264
	mask token index in context... tensor([7])
	decoded target id...  London
	most probable prediction id decoded...  Tokyo


	context... The 2020 Olympics were held in <mask>.
	tokenized_context ids... tensor([[    0,   133,  2760,  4365,    58,   547,    11, 50264,     4,     2]],
       device

In [15]:
score_dict_full

{'distilroberta-base': [{"The 2020 Olympics were held in <mask>. ['Tokyo', 'London']": {'p_true': 0.6587,
    'p_false': 0.0084228515625,
    'p_true - p_false': 0.6502685546875,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in <mask>. ['Tokyo', 'Berlin']": {'p_true': 0.6587,
    'p_false': 0.011688232421875,
    'p_true - p_false': 0.647003173828125,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in <mask>. ['Tokyo', 'Chicago']": {'p_true': 0.6587,
    'p_false': 0.0012617111206054688,
    'p_true - p_false': 0.6574296951293945,
    'p_true > p_false': True}},
  {"Operation Overlord took place in <mask>. ['Normandy', 'Manila']": {'p_true': 0.0015335,
    'p_false': 0.00012105703353881836,
    'p_true - p_false': 0.0014124512672424316,
    'p_true > p_false': True}},
  {"Operation Overlord took place in <mask>. ['Normandy', 'Santiago']": {'p_true': 0.0015335,
    'p_false': 2.866983413696289e-05,
    'p_true - p_false': 0.0015048384666442871,
    'p_t

In [16]:
score_dict_succinct

{'distilroberta-base': [{"The 2020 Olympics were held in <mask>. ['Tokyo', 'London']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in <mask>. ['Tokyo', 'Berlin']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in <mask>. ['Tokyo', 'Chicago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <mask>. ['Normandy', 'Manila']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <mask>. ['Normandy', 'Santiago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <mask>. ['Normandy', 'Baghdad']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <mask>. ['Apple', 'Microsoft']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <mask>. ['Apple', 'Oracle']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <mask>. ['Apple', 'Intel']": {'p_true > p_false': True}}]}

In [17]:
score_dict_summary

{'distilroberta-base': 'This model got 9/9 facts correct'}

### Bert

In [18]:
config = {
    "models": [
        "bert-base-uncased", # 110M params
    ],
    "input_information": {
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false":["London","Berlin", "Chicago"]
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false":["Manila","Santiago", "Baghdad"]
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false":["Microsoft","Oracle", "Intel"]
        }
    },
    "verbosity": True
}


In [19]:
score_dict_full, score_dict_succinct, score_dict_summary = main(config)

CKA for bert-base-uncased
Loading  model...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running comparisons...

	context... The 2020 Olympics were held in [MASK].
	tokenized_context ids... tensor([[  101,  1996, 12609,  3783,  2020,  2218,  1999,   103,  1012,   102]],
       device='cuda:0')
	decoded tokenize_context... [CLS] the 2020 olympics were held in [MASK]. [SEP]
	mask token id... 103
	mask token index in context... tensor([7])
	decoded target id... tokyo
	most probable prediction id decoded... tokyo


	context... The 2020 Olympics were held in [MASK].
	tokenized_context ids... tensor([[  101,  1996, 12609,  3783,  2020,  2218,  1999,   103,  1012,   102]],
       device='cuda:0')
	decoded tokenize_context... [CLS] the 2020 olympics were held in [MASK]. [SEP]
	mask token id... 103
	mask token index in context... tensor([7])
	decoded target id... london
	most probable prediction id decoded... tokyo


	context... The 2020 Olympics were held in [MASK].
	tokenized_context ids... tensor([[  101,  1996, 12609,  3783,  2020,  2218,  1999,   103,  1012,   102]],
       de

In [20]:
score_dict_full

{'bert-base-uncased': [{"The 2020 Olympics were held in [MASK]. ['Tokyo', 'London']": {'p_true': 0.7124,
    'p_false': 0.0040130615234375,
    'p_true - p_false': 0.7083892822265625,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in [MASK]. ['Tokyo', 'Berlin']": {'p_true': 0.7124,
    'p_false': 0.0009756088256835938,
    'p_true - p_false': 0.7114267349243164,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in [MASK]. ['Tokyo', 'Chicago']": {'p_true': 0.7124,
    'p_false': 2.7358531951904297e-05,
    'p_true - p_false': 0.7123749852180481,
    'p_true > p_false': True}},
  {"Operation Overlord took place in [MASK]. ['Normandy', 'Manila']": {'p_true': 0.01952,
    'p_false': 1.3828277587890625e-05,
    'p_true - p_false': 0.01950216293334961,
    'p_true > p_false': True}},
  {"Operation Overlord took place in [MASK]. ['Normandy', 'Santiago']": {'p_true': 0.01952,
    'p_false': 7.331371307373047e-06,
    'p_true - p_false': 0.019508659839630127,
    

In [21]:
score_dict_succinct

{'bert-base-uncased': [{"The 2020 Olympics were held in [MASK]. ['Tokyo', 'London']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in [MASK]. ['Tokyo', 'Berlin']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in [MASK]. ['Tokyo', 'Chicago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in [MASK]. ['Normandy', 'Manila']": {'p_true > p_false': True}},
  {"Operation Overlord took place in [MASK]. ['Normandy', 'Santiago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in [MASK]. ['Normandy', 'Baghdad']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of [MASK]. ['Apple', 'Microsoft']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of [MASK]. ['Apple', 'Oracle']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of [MASK]. ['Apple', 'Intel']": {'p_true > p_false': True}}]}

In [22]:
score_dict_summary

{'bert-base-uncased': 'This model got 9/9 facts correct'}

### gpt2s

In [23]:
config = {
    "models": [
        "distilgpt2",  # 82M params
    ],
    "input_information": {
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false":["London","Berlin", "Chicago"]
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false":["Manila","Santiago", "Baghdad"]
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false":["Microsoft","Oracle", "Intel"]
        }
    },
    "verbosity": True

}


In [24]:
score_dict_full, score_dict_succinct, score_dict_summary = main(config)

CKA for distilgpt2
Loading  model...


Running comparisons...

	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[  464, 12131, 14935,   547,  2714,   287]], device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in
	decoded target id...  Tokyo
	most probable prediction id decoded...  the


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[  464, 12131, 14935,   547,  2714,   287]], device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in
	decoded target id...  London
	most probable prediction id decoded...  the


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[  464, 12131, 14935,   547,  2714,   287]], device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in
	decoded target id...  Tokyo
	most probable prediction id decoded...  the


	context... The 2020 Olympics were held in
	tokenized_context ids... tensor([[  464, 12131, 14935,   547,  2714,   287]], device='cuda:0')
	decoded

In [25]:
score_dict_full

{'distilgpt2': [{"The 2020 Olympics were held in ['Tokyo', 'London']": {'p_true': 0.01068,
    'p_false': 0.0478515625,
    'p_true - p_false': -0.03717041015625,
    'p_true > p_false': False}},
  {"The 2020 Olympics were held in ['Tokyo', 'Berlin']": {'p_true': 0.01068,
    'p_false': 0.006084442138671875,
    'p_true - p_false': 0.004596710205078125,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Chicago']": {'p_true': 0.01068,
    'p_false': 0.0015382766723632812,
    'p_true - p_false': 0.009142875671386719,
    'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Manila']": {'p_true': 0.0007377,
    'p_false': 0.00019848346710205078,
    'p_true - p_false': 0.0005391836166381836,
    'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Santiago']": {'p_true': 0.0007377,
    'p_false': 6.860494613647461e-05,
    'p_true - p_false': 0.0006690621376037598,
    'p_true > p_false': True}},
  {"Operation

In [26]:
score_dict_succinct

{'distilgpt2': [{"The 2020 Olympics were held in ['Tokyo', 'London']": {'p_true > p_false': False}},
  {"The 2020 Olympics were held in ['Tokyo', 'Berlin']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in ['Tokyo', 'Chicago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Manila']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Santiago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in ['Normandy', 'Baghdad']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Microsoft']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Oracle']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of ['Apple', 'Intel']": {'p_true > p_false': True}}]}

In [27]:
score_dict_summary

{'distilgpt2': 'This model got 8/9 facts correct'}

### Google/flans

In [28]:
config = {
    "models": [
        "google/flan-t5-small",  # 80M params
    ],
    "input_information": {
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false":["London","Berlin", "Chicago"]
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false":["Manila","Santiago", "Baghdad"]
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false":["Microsoft","Oracle", "Intel"]
        }
    },
    "verbosity": True

}


In [29]:
score_dict_full, score_dict_succinct, score_dict_summary = main(config)

CKA for google/flan-t5-small
Loading  model...


Running comparisons...

	context... The 2020 Olympics were held in <extra_id_0>.
	tokenized_context ids... tensor([[   37,  6503, 17793,   130,  1213,    16, 32099,     3,     5,     1]],
       device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in<extra_id_0>.</s>
	decoded target id... Tokyo
	most probable prediction id decoded... a


	context... The 2020 Olympics were held in <extra_id_0>.
	tokenized_context ids... tensor([[   37,  6503, 17793,   130,  1213,    16, 32099,     3,     5,     1]],
       device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in<extra_id_0>.</s>
	decoded target id... London
	most probable prediction id decoded... a


	context... The 2020 Olympics were held in <extra_id_0>.
	tokenized_context ids... tensor([[   37,  6503, 17793,   130,  1213,    16, 32099,     3,     5,     1]],
       device='cuda:0')
	decoded tokenized_context... The 2020 Olympics were held in<extra_id_0>.</s>
	decoded target id... Tokyo
	most

In [30]:
score_dict_full

{'google/flan-t5-small': [{"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'London']": {'p_true': 3.46e-06,
    'p_false': 2.9742717742919922e-05,
    'p_true - p_false': -2.6285648345947266e-05,
    'p_true > p_false': False}},
  {"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'Berlin']": {'p_true': 3.46e-06,
    'p_false': 3.5762786865234375e-07,
    'p_true - p_false': 3.0994415283203125e-06,
    'p_true > p_false': True}},
  {"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'Chicago']": {'p_true': 3.46e-06,
    'p_false': 2.384185791015625e-07,
    'p_true - p_false': 3.2186508178710938e-06,
    'p_true > p_false': True}},
  {"Operation Overlord took place in <extra_id_0>. ['Normandy', 'Manila']": {'p_true': 2.06e-05,
    'p_false': 1.0728836059570312e-06,
    'p_true - p_false': 1.9550323486328125e-05,
    'p_true > p_false': True}},
  {"Operation Overlord took place in <extra_id_0>. ['Normandy', 'Santiago']": {'p_true': 2.06e-05,
    'p_false': 2.980232238

In [31]:
score_dict_succinct

{'google/flan-t5-small': [{"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'London']": {'p_true > p_false': False}},
  {"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'Berlin']": {'p_true > p_false': True}},
  {"The 2020 Olympics were held in <extra_id_0>. ['Tokyo', 'Chicago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <extra_id_0>. ['Normandy', 'Manila']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <extra_id_0>. ['Normandy', 'Santiago']": {'p_true > p_false': True}},
  {"Operation Overlord took place in <extra_id_0>. ['Normandy', 'Baghdad']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <extra_id_0>. ['Apple', 'Microsoft']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <extra_id_0>. ['Apple', 'Oracle']": {'p_true > p_false': True}},
  {"Steve Jobs is the founder of <extra_id_0>. ['Apple', 'Intel']": {'p_true > p_false': True}}]}

In [32]:
score_dict_summary

{'google/flan-t5-small': 'This model got 8/9 facts correct'}